In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using KrylovKit
using SparseArrays
using LinearAlgebra
using Plots
using LaTeXStrings
using FFTW
using LsqFit

using QuantumModels
using MatrixProductStateTools
using ExactDiagonalizationTools
using SymTensors
using GaussianFermions
using GutzwillerMPS

In [ ]:
ly,lx=4,12
L = 1 * lx
m = 30
hamil = generatebdg(triangularhopping((ly,lx), 1.0, 1., 1., 1., boundary=(:PBC, :OBC)))
cm = correlationmatrix(hamil, div(L,2))
fgs = corrmat2gmps(cm)
mps = gmps2mps(fgs, m, symmetry=U1)

sz, sp, sm = spinoperators(1/2, symmetry=U1)

fhop_pm = measure(mps, sp, sm)
#fhop_mp = measure(mps, sm, sp)
fhop_zz = measure(mps, sz, sz)
indexes = [(1,l+1) for l=1:lx-1]

p = plot(xaxis=(L"$x$"), yaxis=(L"$\langle s^\alpha_1 s^\beta_x\rangle$"), legend=:bottomright)
plot!(p, [2:lx], [fhop_zz[i] for i in indexes], marker=(:circle), label=L"\alpha=z, \beta=z")
plot!(p, [2:lx], [fhop_pm[i] for i in indexes], marker=(:circle), label=L"\alpha=+, \beta=-")
#plot!(p, [2:lx], [fhop_mp[i] for i in indexes], marker=(:circle), label=L"\alpha=-, \beta=+")

In [ ]:
mpsgutz, ortho, ets = GutzwillerMPS._zipandgutzwiller_F23_analysis!(mps, mps, maxdim=20);

In [ ]:
plt = plot()
plot!(plt, log.(abs.(1 .-ortho))[2:L-2])
plot!(plt, log.(abs.(1 .-ets))[2:L-2])

In [ ]:
ortho

In [ ]:
ortho

In [ ]:
prodmps = GutzwillerMPS._tensorproductzip!(mps, mps, maxdim=400);
#normalize!(gutz1)
gutz2 = zipandgutzwiller!(mps, mps, maxdim=200);

In [ ]:
vne = entanglemententropy(mps)
vne1 = entanglemententropy(prodmps)
#vne2 = entanglemententropy(gutz2);

In [ ]:
plt=plot()
plot!(plt, vne)
plot!(plt, vne1)
#plot!(plt, vne2)

In [ ]:
bonddims(prodmps)

In [ ]:
G = fill(one(Float64), zero(U1),
             (U1Space([-1=>1, 1=>1]),
              dual(U1Space([-1=>1, 0=>2, 1=>1]))
              ))


C = contract(G, (2, -1), prodmps.As[lx], (1, -1, 3))
    

In [ ]:
n=lx-1
A = prodmps.As[n]
C = contract(G, (2, -1), prodmps.As[n], (1, -1, 3))
isrightisometry(fuselegs(C, 2, 2, true))
#isleftisometry(fuselegs(A, 1, 2))

In [ ]:
center_at!(prodmps, 1);


In [ ]:
G

In [ ]:
R = contract(
    contract(prodmps.As[lx], (1, -1, 3), G, (2, -1)), (1, -1, -2), 
    contract(dual(prodmps.As[lx]), (1, -1, 3), G, (2, -1)), (2, -1, -2))

l = lx-1
A = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
B = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
R = contract(contract(A, (1,2,-1), R, (-1,3)), (1,-1,-2), B, (2,-1,-2))

l = lx-2
A = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
B = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
R = contract(contract(A, (1,2,-1), R, (-1,3)), (1,-1,-2), B, (2,-1,-2))

l = lx-3
A = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
B = contract(prodmps.As[l], (1, -1, 3), G, (2, -1))
R = contract(contract(A, (1,2,-1), R, (-1,3)), (1,-1,-2), B, (2,-1,-2))